# Assignment 1 Part 2
Simon Jasansky, Akos Engelmann, Ipek Cakin, Gergely Parady

In [2]:
import pandas as pd
import numpy as np
from rdflib import URIRef, BNode, Literal, Namespace
from rdflib.namespace import XSD, RDF, RDFS
from rdflib import Graph
from urllib.parse import quote

## We split the original music.csv dataframe into three: 
1. Music
2. Artists
3. Genres

In [3]:
music = pd.read_csv("data/music.csv")
music

,index,title,artist,top genre,year
0,0,"Hey, Soul Sister",Train,neo mellow,2010
1,1,Love The Way You Lie,Eminem,detroit hip hop,2010
2,2,TiK ToK,Kesha,dance pop,2010
3,3,Bad Romance,Lady Gaga,dance pop,2010
4,4,Just the Way You Are,Bruno Mars,pop,2010
...,...,...,...,...,...
598,598,Find U Again (feat. Camila Cabello),Mark Ronson,dance pop,2019
599,599,Cross Me (feat. Chance the Rapper & PnB Rock),Ed Sheeran,pop,2019
600,600,"No Brainer (feat. Justin Bieber, Chance the Ra...",DJ Khaled,dance pop,2019
601,601,Nothing Breaks Like a Heart (feat. Miley Cyrus),Mark Ronson,dance pop,2019


In [9]:
artists = music.artist.unique()
genre = music["top genre"].unique()
artists
genre

array(['neo mellow', 'detroit hip hop', 'dance pop', 'pop',
       'canadian pop', 'hip pop', 'barbadian pop', 'atl hip hop',
       'australian pop', 'indie pop', 'art pop', 'colombian pop',
       'big room', 'british soul', 'chicago rap', 'acoustic pop',
       'permanent wave', 'boy band', 'baroque pop', 'celtic rock',
       'electro', 'complextro', 'canadian hip hop', 'candy pop',
       'alaska indie', 'folk-pop', 'metropopolis', 'house',
       'australian hip hop', 'electropop', 'australian dance',
       'hollywood', 'canadian contemporary r&b',
       'irish singer-songwriter', 'tropical house', 'belgian edm',
       'french indie pop', 'hip hop', 'danish pop', 'latin',
       'canadian latin', 'electronic trap', 'edm', 'electro house',
       'downtempo', 'brostep', 'contemporary country', 'moroccan pop',
       'escape room', 'alternative r&b'], dtype=object)